In [2]:
from numpy import array, ndarray
from time import time
from itertools import product

import code.profiles.generation as gen
from code.profiles.definitions import Profile, Voter, Candidate, notVCR33
from code.profiles.vcrDetection import detectVCRProperty, detectCRProperty, detectVRProperty
from code.profiles.vcrDomain import isVCR

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext # sc = SparkContext.getOrCreate()
spark

In [ ]:
def run(C=3, V=3):
    candidatesIds = ["A", "B", "C", "D", "E", "F", "G"][:C]
    votersIds = ["0", "1", "2", "3", "4", "5", "6"][:V]

    profilesNP = sc.parallelize(product([0, 1], repeat=C)) \
        .map(lambda singleVote: product(singleVote, repeat=V)) \
        .map(lambda profile: array(profile))

    # profilesRDD = sc.parallelize(gen.parallelProfileGeneration(cpu=16, candidatesNumber=C, voterNumber=V))
    profilesRDD = profilesNP

    print("TOTAL : ", profilesRDD.count())

    # p.A, (status, resDict)
    vcrProfilesRDD = profilesRDD \
        .map(lambda p: (p,detectVCRProperty(A=p, C=candidatesIds[:C], V=votersIds[:V]))) \
        .filter(lambda pRes: pRes[1][0] == 2)

    print("VCR : ", vcrProfilesRDD.count())
    
    vcrNCOPProfilesRDD = vcrProfilesRDD \
        .filter(lambda pRes: not detectCRProperty(A=pRes[0], C=candidatesIds, V=votersIds)) \
        .filter(lambda pRes: not detectVRProperty(A=pRes[0], C=candidatesIds, V=votersIds)) \
        .map(lambda pRes: Profile.fromILPRes(pRes[0], pRes[1][1], candidatesIds, votersIds)) \
        .map(lambda profile: profile.asNumpy())

    print("NCOP VCR : ", vcrNCOPProfilesRDD.count())
    return vcrNCOPProfilesRDD

In [ ]:
start = time()
vcrNCOPProfiles = run(3,3)
end = time()
end - start

In [ ]:
vcrNCOPProfiles.take(3)

import numpy as np
a = notVCR33().asNumpy()
bb = Profile.fromNumpy(a).asNumpy().tobytes()
np.frombuffer(bb)[20:].astype(int)